# Deep learning

In this file, we try to use the ANN, combined with other approaches like (de)centralized and seasonality aggregation, for predicting weekly sales. 

In [56]:
import pandas as pd
sales=pd.read_csv("E:/GitHubActivities/rep1-Analysis on retail demand prediction/data/data_processed.csv")
NW = 98 #number of weeks to be considered in the dataset

# Centralized and decentralized approaches

There are two general approaches to predict the demand of products. The first is to consider a single predictor for all product items, called centralized method, and the other approach is to consider products separately, called decentralized method.


In [57]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import r2_score, mean_squared_error

ANN = keras.models.Sequential([
    keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(1, activation='LeakyReLU')
])

ANN.compile(optimizer=tf.compat.v1.train.AdamOptimizer(), 
              loss='mean_squared_error',
              metrics=['mean_squared_error'])


res=pd.DataFrame(index=['R2'])

## Structuring the dataset

Before proceeding to the approaches mentioned above, we need to split our data into train and test datasets for each specific product.

In [58]:
skuSet = list(sales.sku.unique())
skuData = {}
colnames = [i for i in sales.columns if i not in ["week","weekly_sales","sku"]]
for i in skuSet:
  df_i = sales[sales.sku == i]
  skuData[i] = {'X': df_i[:NW][colnames].values,
                'y': df_i[:NW]['weekly_sales'].values}
    
X_dict = {}
y_dict = {}

y_test = []
y_train = []

for i in skuSet:
  
  X_train_i,X_test_i = train_test_split(skuData[i]["X"], shuffle=False, train_size=0.7) #split for X
  y_train_i,y_test_i = train_test_split(skuData[i]["y"], shuffle=False, train_size=0.7) #split for y 

  X_dict[i] = {'train': X_train_i, 'test': X_test_i} #filling dictionary
  y_dict[i] = {'train': y_train_i, 'test': y_test_i}

  y_test += list(y_test_i) 
  y_train += list(y_train_i) 

## Centralized method

 Once the train and test datasets are created for each product, we combine them to deploy our centralized solution method and evaluate it.

In [59]:
X_cen_train = X_dict[skuSet[0]]['train'] #initialization with item 0
X_cen_test = X_dict[skuSet[0]]['test']

for i in skuSet[1:]: #Iteration over items
    X_cen_train = np.concatenate((X_cen_train, X_dict[i]['train']), axis = 0) #Bringing together the training set
    X_cen_test = np.concatenate((X_cen_test, X_dict[i]['test']), axis = 0)

X_cen_train = np.asarray(X_cen_train)
X_cen_test = np.asarray(X_cen_test)

y_test = np.asarray(y_test)
y_train = np.asarray(y_train)

model_cen = ANN
model_cen.fit(X_cen_train,y_train,batch_size=20,epochs=500,validation_split=0)

y_pred = model_cen.predict(X_cen_test)

print('Centralized method with linear regression R2:',
      round(r2_score(y_test, np.array(y_pred)),3))  
print('Centralized method with linear regression MSE:',
      round(mean_squared_error(y_test, np.array(y_pred)),3))

res['Centralized(DL)']=[r2_score(y_test, model_cen.predict(X_cen_test))]

Epoch 1/500
150/150 [==============================] - 1s 2ms/step - loss: 78131.1953 - mean_squared_error: 78131.1797
Epoch 2/500
150/150 [==============================] - 0s 2ms/step - loss: 77838.3438 - mean_squared_error: 77838.3438
Epoch 3/500
150/150 [==============================] - 0s 2ms/step - loss: 77412.2969 - mean_squared_error: 77412.2969
Epoch 4/500
150/150 [==============================] - 0s 2ms/step - loss: 76658.8516 - mean_squared_error: 76658.8516
Epoch 5/500
150/150 [==============================] - 0s 3ms/step - loss: 75603.1328 - mean_squared_error: 75603.1328
Epoch 6/500
150/150 [==============================] - 0s 3ms/step - loss: 74334.3828 - mean_squared_error: 74334.3828
Epoch 7/500
150/150 [==============================] - 0s 2ms/step - loss: 72948.2422 - mean_squared_error: 72948.2422
Epoch 8/500
150/150 [==============================] - 0s 2ms/step - loss: 71632.5547 - mean_squared_error: 71632.5547
Epoch 9/500
150/150 [===========================

150/150 [==============================] - 0s 1ms/step - loss: 34177.5312 - mean_squared_error: 34177.5312
Epoch 138/500
150/150 [==============================] - 0s 1ms/step - loss: 34156.1992 - mean_squared_error: 34156.1992
Epoch 139/500
150/150 [==============================] - 0s 1ms/step - loss: 34042.9180 - mean_squared_error: 34042.9141
Epoch 140/500
150/150 [==============================] - 0s 1ms/step - loss: 33968.1641 - mean_squared_error: 33968.1641
Epoch 141/500
150/150 [==============================] - 0s 1ms/step - loss: 33932.8164 - mean_squared_error: 33932.8164
Epoch 142/500
150/150 [==============================] - 0s 1ms/step - loss: 33864.4531 - mean_squared_error: 33864.4531
Epoch 143/500
150/150 [==============================] - 0s 1ms/step - loss: 33807.7930 - mean_squared_error: 33807.7930
Epoch 144/500
150/150 [==============================] - 0s 1ms/step - loss: 33779.2500 - mean_squared_error: 33779.2500
Epoch 145/500
150/150 [=======================

150/150 [==============================] - 0s 1ms/step - loss: 29779.8555 - mean_squared_error: 29779.8555
Epoch 272/500
150/150 [==============================] - 0s 1ms/step - loss: 29888.0801 - mean_squared_error: 29888.0801
Epoch 273/500
150/150 [==============================] - 0s 1ms/step - loss: 29797.4180 - mean_squared_error: 29797.4180
Epoch 274/500
150/150 [==============================] - 0s 1ms/step - loss: 29747.4258 - mean_squared_error: 29747.4258
Epoch 275/500
150/150 [==============================] - 0s 2ms/step - loss: 29687.5879 - mean_squared_error: 29687.5879
Epoch 276/500
150/150 [==============================] - 0s 1ms/step - loss: 29683.4688 - mean_squared_error: 29683.4688
Epoch 277/500
150/150 [==============================] - 0s 1ms/step - loss: 29636.7949 - mean_squared_error: 29636.7949
Epoch 278/500
150/150 [==============================] - 0s 1ms/step - loss: 29628.6875 - mean_squared_error: 29628.6875
Epoch 279/500
150/150 [=======================

150/150 [==============================] - 0s 2ms/step - loss: 27412.5859 - mean_squared_error: 27412.5859
Epoch 406/500
150/150 [==============================] - 0s 2ms/step - loss: 27546.6602 - mean_squared_error: 27546.6602
Epoch 407/500
150/150 [==============================] - 0s 2ms/step - loss: 27455.2246 - mean_squared_error: 27455.2246
Epoch 408/500
150/150 [==============================] - 0s 2ms/step - loss: 27510.9785 - mean_squared_error: 27510.9785
Epoch 409/500
150/150 [==============================] - 0s 1ms/step - loss: 27476.4551 - mean_squared_error: 27476.4551
Epoch 410/500
150/150 [==============================] - 0s 2ms/step - loss: 27439.6719 - mean_squared_error: 27439.6719
Epoch 411/500
150/150 [==============================] - 0s 1ms/step - loss: 27403.8359 - mean_squared_error: 27403.8359
Epoch 412/500
150/150 [==============================] - 0s 2ms/step - loss: 27368.5215 - mean_squared_error: 27368.5234
Epoch 413/500
150/150 [=======================

## Decentralized mehod

In this subsection, a dictionary of prediction mehods is created for each productn and the total accuracy of it is caculated then.


In [60]:
y_pred = []
skuModels = {}

for i in skuSet:
 #one model for each item, fitted on training set
 model_i = ANN
 skuModels[i] = model_i.fit(X_dict[i]['train'],y_dict[i]['train'],batch_size=20,epochs=50,validation_split=0)

 #compute and concatenate prediction of the model i on item i
 y_pred += list(model_i.predict(X_dict[i]['test']))


#computing overall performance metrics on y_pred and y_test:
print('Decentralized method with linear regression R2:',round(r2_score(y_test, np.array(y_pred)),3))
print('Decentralized method with linear regression MSE:', round(mean_squared_error(y_test, np.array(y_pred)),3))

res['Decentralized(DL)']=[r2_score(y_test, np.array(y_pred))]

Epoch 1/50
4/4 [==============================] - 0s 2ms/step - loss: 5917.7271 - mean_squared_error: 5917.7271
Epoch 2/50
4/4 [==============================] - 0s 2ms/step - loss: 5313.5420 - mean_squared_error: 5313.5420
Epoch 3/50
4/4 [==============================] - 0s 2ms/step - loss: 4625.6572 - mean_squared_error: 4625.6572
Epoch 4/50
4/4 [==============================] - 0s 2ms/step - loss: 3963.3860 - mean_squared_error: 3963.3860
Epoch 5/50
4/4 [==============================] - 0s 2ms/step - loss: 3530.8071 - mean_squared_error: 3530.8071
Epoch 6/50
4/4 [==============================] - 0s 3ms/step - loss: 3190.7246 - mean_squared_error: 3190.7246
Epoch 7/50
4/4 [==============================] - 0s 3ms/step - loss: 3015.8074 - mean_squared_error: 3015.8074
Epoch 8/50
4/4 [==============================] - 0s 2ms/step - loss: 2851.9971 - mean_squared_error: 2851.9971
Epoch 9/50
4/4 [==============================] - 0s 2ms/step - loss: 2729.0859 - mean_squared_error: 27

4/4 [==============================] - 0s 3ms/step - loss: 99.6900 - mean_squared_error: 99.6900
Epoch 25/50
4/4 [==============================] - 0s 2ms/step - loss: 100.1855 - mean_squared_error: 100.1855
Epoch 26/50
4/4 [==============================] - 0s 2ms/step - loss: 99.7064 - mean_squared_error: 99.7064
Epoch 27/50
4/4 [==============================] - 0s 2ms/step - loss: 99.3644 - mean_squared_error: 99.3644
Epoch 28/50
4/4 [==============================] - 0s 2ms/step - loss: 99.2495 - mean_squared_error: 99.2495
Epoch 29/50
4/4 [==============================] - 0s 2ms/step - loss: 99.0210 - mean_squared_error: 99.0210
Epoch 30/50
4/4 [==============================] - 0s 2ms/step - loss: 101.0378 - mean_squared_error: 101.0378
Epoch 31/50
4/4 [==============================] - 0s 2ms/step - loss: 99.6386 - mean_squared_error: 99.6386
Epoch 32/50
4/4 [==============================] - 0s 2ms/step - loss: 99.2998 - mean_squared_error: 99.2998
Epoch 33/50
4/4 [==========

4/4 [==============================] - 0s 2ms/step - loss: 43.6877 - mean_squared_error: 43.6877
Epoch 24/50
4/4 [==============================] - 0s 2ms/step - loss: 43.5482 - mean_squared_error: 43.5482
Epoch 25/50
4/4 [==============================] - 0s 2ms/step - loss: 43.5299 - mean_squared_error: 43.5299
Epoch 26/50
4/4 [==============================] - 0s 2ms/step - loss: 43.5369 - mean_squared_error: 43.5369
Epoch 27/50
4/4 [==============================] - 0s 2ms/step - loss: 43.5541 - mean_squared_error: 43.5541
Epoch 28/50
4/4 [==============================] - 0s 2ms/step - loss: 43.5538 - mean_squared_error: 43.5538
Epoch 29/50
4/4 [==============================] - 0s 2ms/step - loss: 43.5341 - mean_squared_error: 43.5341
Epoch 30/50
4/4 [==============================] - 0s 2ms/step - loss: 43.6475 - mean_squared_error: 43.6475
Epoch 31/50
4/4 [==============================] - 0s 3ms/step - loss: 43.5589 - mean_squared_error: 43.5589
Epoch 32/50
4/4 [==============

4/4 [==============================] - 0s 2ms/step - loss: 203.5744 - mean_squared_error: 203.5744
Epoch 21/50
4/4 [==============================] - 0s 2ms/step - loss: 208.5433 - mean_squared_error: 208.5433
Epoch 22/50
4/4 [==============================] - 0s 2ms/step - loss: 209.4547 - mean_squared_error: 209.4547
Epoch 23/50
4/4 [==============================] - 0s 2ms/step - loss: 208.9282 - mean_squared_error: 208.9282
Epoch 24/50
4/4 [==============================] - 0s 2ms/step - loss: 199.0256 - mean_squared_error: 199.0256
Epoch 25/50
4/4 [==============================] - 0s 2ms/step - loss: 209.9041 - mean_squared_error: 209.9041
Epoch 26/50
4/4 [==============================] - 0s 2ms/step - loss: 206.4704 - mean_squared_error: 206.4704
Epoch 27/50
4/4 [==============================] - 0s 2ms/step - loss: 211.3478 - mean_squared_error: 211.3478
Epoch 28/50
4/4 [==============================] - 0s 2ms/step - loss: 213.2039 - mean_squared_error: 213.2039
Epoch 29/50
4

4/4 [==============================] - 0s 4ms/step - loss: 2178.1162 - mean_squared_error: 2178.1162
Epoch 18/50
4/4 [==============================] - 0s 4ms/step - loss: 2174.5652 - mean_squared_error: 2174.5652
Epoch 19/50
4/4 [==============================] - 0s 4ms/step - loss: 2183.1584 - mean_squared_error: 2183.1584
Epoch 20/50
4/4 [==============================] - 0s 3ms/step - loss: 2174.1863 - mean_squared_error: 2174.1863
Epoch 21/50
4/4 [==============================] - 0s 3ms/step - loss: 2174.3184 - mean_squared_error: 2174.3184
Epoch 22/50
4/4 [==============================] - 0s 4ms/step - loss: 2179.7922 - mean_squared_error: 2179.7922
Epoch 23/50
4/4 [==============================] - 0s 4ms/step - loss: 2177.6714 - mean_squared_error: 2177.6714
Epoch 24/50
4/4 [==============================] - 0s 3ms/step - loss: 2176.3687 - mean_squared_error: 2176.3687
Epoch 25/50
4/4 [==============================] - 0s 2ms/step - loss: 2175.0620 - mean_squared_error: 2175.

...

4/4 [==============================] - 0s 2ms/step - loss: 467.7911 - mean_squared_error: 467.7911
Epoch 29/50
4/4 [==============================] - 0s 2ms/step - loss: 467.5332 - mean_squared_error: 467.5332
Epoch 30/50
4/4 [==============================] - 0s 2ms/step - loss: 467.8180 - mean_squared_error: 467.8180
Epoch 31/50
4/4 [==============================] - 0s 2ms/step - loss: 468.6762 - mean_squared_error: 468.6762
Epoch 32/50
4/4 [==============================] - 0s 2ms/step - loss: 468.6117 - mean_squared_error: 468.6117
Epoch 33/50
4/4 [==============================] - 0s 2ms/step - loss: 468.4803 - mean_squared_error: 468.4803
Epoch 34/50
4/4 [==============================] - 0s 2ms/step - loss: 468.9837 - mean_squared_error: 468.9837
Epoch 35/50
4/4 [==============================] - 0s 2ms/step - loss: 467.6575 - mean_squared_error: 467.6575
Epoch 36/50
4/4 [==============================] - 0s 2ms/step - loss: 467.1274 - mean_squared_error: 467.1274
Epoch 37/50
4

4/4 [==============================] - 0s 2ms/step - loss: 42099.5742 - mean_squared_error: 42099.5742
Epoch 22/50
4/4 [==============================] - 0s 2ms/step - loss: 41841.5078 - mean_squared_error: 41841.5078
Epoch 23/50
4/4 [==============================] - 0s 2ms/step - loss: 41547.4375 - mean_squared_error: 41547.4375
Epoch 24/50
4/4 [==============================] - 0s 2ms/step - loss: 41302.0938 - mean_squared_error: 41302.0938
Epoch 25/50
4/4 [==============================] - 0s 2ms/step - loss: 41215.7656 - mean_squared_error: 41215.7656
Epoch 26/50
4/4 [==============================] - 0s 2ms/step - loss: 40957.7773 - mean_squared_error: 40957.7773
Epoch 27/50
4/4 [==============================] - 0s 2ms/step - loss: 40692.3125 - mean_squared_error: 40692.3125
Epoch 28/50
4/4 [==============================] - 0s 2ms/step - loss: 40539.0430 - mean_squared_error: 40539.0430
Epoch 29/50
4/4 [==============================] - 0s 2ms/step - loss: 40370.0977 - mean_squ

4/4 [==============================] - 0s 2ms/step - loss: 61.2277 - mean_squared_error: 61.2277
Epoch 17/50
4/4 [==============================] - 0s 2ms/step - loss: 61.7145 - mean_squared_error: 61.7145
Epoch 18/50
4/4 [==============================] - 0s 2ms/step - loss: 61.7839 - mean_squared_error: 61.7839
Epoch 19/50
4/4 [==============================] - 0s 2ms/step - loss: 61.5426 - mean_squared_error: 61.5426
Epoch 20/50
4/4 [==============================] - 0s 2ms/step - loss: 60.9044 - mean_squared_error: 60.9044
Epoch 21/50
4/4 [==============================] - 0s 2ms/step - loss: 60.8387 - mean_squared_error: 60.8387
Epoch 22/50
4/4 [==============================] - 0s 2ms/step - loss: 60.5352 - mean_squared_error: 60.5352
Epoch 23/50
4/4 [==============================] - 0s 2ms/step - loss: 60.5633 - mean_squared_error: 60.5633
Epoch 24/50
4/4 [==============================] - 0s 2ms/step - loss: 60.5823 - mean_squared_error: 60.5823
Epoch 25/50
4/4 [==============

4/4 [==============================] - 0s 3ms/step - loss: 9132.5293 - mean_squared_error: 9132.5293
Epoch 15/50
4/4 [==============================] - 0s 2ms/step - loss: 9067.7373 - mean_squared_error: 9067.7373
Epoch 16/50
4/4 [==============================] - 0s 2ms/step - loss: 9005.4678 - mean_squared_error: 9005.4678
Epoch 17/50
4/4 [==============================] - 0s 2ms/step - loss: 8929.6992 - mean_squared_error: 8929.6992
Epoch 18/50
4/4 [==============================] - 0s 2ms/step - loss: 8869.9492 - mean_squared_error: 8869.9492
Epoch 19/50
4/4 [==============================] - 0s 2ms/step - loss: 8810.8105 - mean_squared_error: 8810.8105
Epoch 20/50
4/4 [==============================] - 0s 2ms/step - loss: 8750.4746 - mean_squared_error: 8750.4746
Epoch 21/50
4/4 [==============================] - 0s 2ms/step - loss: 8692.9795 - mean_squared_error: 8692.9795
Epoch 22/50
4/4 [==============================] - 0s 2ms/step - loss: 8631.1436 - mean_squared_error: 8631.

4/4 [==============================] - 0s 2ms/step - loss: 86.8784 - mean_squared_error: 86.8784
Epoch 11/50
4/4 [==============================] - 0s 2ms/step - loss: 70.6752 - mean_squared_error: 70.6752
Epoch 12/50
4/4 [==============================] - 0s 2ms/step - loss: 83.7557 - mean_squared_error: 83.7557
Epoch 13/50
4/4 [==============================] - 0s 2ms/step - loss: 70.4113 - mean_squared_error: 70.4113
Epoch 14/50
4/4 [==============================] - 0s 2ms/step - loss: 69.7994 - mean_squared_error: 69.7994
Epoch 15/50
4/4 [==============================] - 0s 3ms/step - loss: 67.8070 - mean_squared_error: 67.8070
Epoch 16/50
4/4 [==============================] - 0s 2ms/step - loss: 64.2794 - mean_squared_error: 64.2794
Epoch 17/50
4/4 [==============================] - 0s 2ms/step - loss: 69.8459 - mean_squared_error: 69.8459
Epoch 18/50
4/4 [==============================] - 0s 2ms/step - loss: 66.3256 - mean_squared_error: 66.3256
Epoch 19/50
4/4 [==============

# Prediction with aggregated seasonality

A common approach in retail is to consider different coefficients for item products. In this section, all the features, except for the seasonality, are cosidered at product item level inside the dataset. This method is called feature-fixed effect. Note that this method may result in a better prediction for the centralized method, but the decentraized method already considers the dataset at the product item levels. As a result, the prediction accuracy of the decentralized method does not differ from the subsection 2.3.

## Structuring the dataset

In [61]:
sales_fe_sku = sales.copy()
sales_fe_sku = pd.get_dummies(data=sales_fe_sku, columns=['sku'])
sales_fe_sku["sku"] = sales["sku"] 


colnames_to_fix = [i for i in sales.columns if i not in ["week","weekly_sales","sku",
                                                         'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
                                                         'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 
                                                         'month_12']]

sales_seasonality = sales_fe_sku.copy()

for feature in colnames_to_fix:
  for i in range(1,45):
    sales_seasonality[str(feature)+"_fixed_effect_"+str(i)] = sales_seasonality[feature]*sales_seasonality["sku_"+str(i)]

skuSet = list(sales.sku.unique()) #the SKU numbers do not change
skuData = {}
colnames = [i for i in sales_seasonality.columns if i not in ["week","weekly_sales","sku"] and i not in colnames_to_fix]
for i in skuSet:
  df_i = sales_seasonality[sales_seasonality.sku == i]
  skuData[i] = {'X': df_i[:NW][colnames].values,
                'y': df_i[:NW]['weekly_sales'].values}
    


X_dict = {}
y_dict = {}

y_test = []
y_train = []

for i in skuSet:
  
  X_train_i,X_test_i = train_test_split(skuData[i]["X"], shuffle=False, train_size=0.7) #split for X
  y_train_i,y_test_i = train_test_split(skuData[i]["y"], shuffle=False, train_size=0.7) #split for y 

  X_dict[i] = {'train': X_train_i, 'test': X_test_i} #filling dictionary
  y_dict[i] = {'train': y_train_i, 'test': y_test_i}

  y_test += list(y_test_i) 
  y_train += list(y_train_i) 



C:\Users\NOOR~1\AppData\Local\Temp/ipykernel_16680/1985272825.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_seasonality[str(feature)+"_fixed_effect_"+str(i)] = sales_seasonality[feature]*sales_seasonality["sku_"+str(i)]


## Centralized method

In [62]:
X_cen_train = X_dict[skuSet[0]]['train'] #initialization with item 0
X_cen_test = X_dict[skuSet[0]]['test']

for i in skuSet[1:]: #Iteration over items
    X_cen_train = np.concatenate((X_cen_train, X_dict[i]['train']), axis = 0) #Bringing together the training set
    X_cen_test = np.concatenate((X_cen_test, X_dict[i]['test']), axis = 0)

X_cen_train = np.asarray(X_cen_train)
X_cen_test = np.asarray(X_cen_test)

y_test = np.asarray(y_test)
y_train = np.asarray(y_train)

ANN = keras.models.Sequential([
    keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(1, activation='LeakyReLU')
])

ANN.compile(optimizer=tf.compat.v1.train.AdamOptimizer(), 
              loss='mean_squared_error',
              metrics=['mean_squared_error'])


model_cen = ANN
model_cen.fit(X_cen_train,y_train,batch_size=20,epochs=500,validation_split=0)

y_pred = model_cen.predict(X_cen_test)

print('Centralized method with linear regression R2:',
      round(r2_score(y_test, np.array(y_pred)),3))  
print('Centralized method with linear regression MSE:',
      round(mean_squared_error(y_test, np.array(y_pred)),3))

res['Centralized(SA-DL)']=[r2_score(y_test, model_cen.predict(X_cen_test))]

Epoch 1/500
150/150 [==============================] - 1s 2ms/step - loss: 78464.5469 - mean_squared_error: 78464.5469
Epoch 2/500
150/150 [==============================] - 0s 2ms/step - loss: 76699.1172 - mean_squared_error: 76699.1172
Epoch 3/500
150/150 [==============================] - 0s 2ms/step - loss: 73799.2266 - mean_squared_error: 73799.2266
Epoch 4/500
150/150 [==============================] - 0s 2ms/step - loss: 69875.8359 - mean_squared_error: 69875.8359
Epoch 5/500
150/150 [==============================] - 0s 2ms/step - loss: 66610.6172 - mean_squared_error: 66610.6172
Epoch 6/500
150/150 [==============================] - 0s 2ms/step - loss: 64393.2500 - mean_squared_error: 64393.2500
Epoch 7/500
150/150 [==============================] - 0s 2ms/step - loss: 62758.5352 - mean_squared_error: 62758.5352
Epoch 8/500
150/150 [==============================] - 0s 2ms/step - loss: 61435.8438 - mean_squared_error: 61435.8398
Epoch 9/500
150/150 [===========================

150/150 [==============================] - 0s 2ms/step - loss: 22861.7031 - mean_squared_error: 22861.7031
Epoch 138/500
150/150 [==============================] - 0s 2ms/step - loss: 22697.3047 - mean_squared_error: 22697.3047
Epoch 139/500
150/150 [==============================] - 0s 2ms/step - loss: 22552.1250 - mean_squared_error: 22552.1250
Epoch 140/500
150/150 [==============================] - 0s 2ms/step - loss: 22544.5508 - mean_squared_error: 22544.5508
Epoch 141/500
150/150 [==============================] - 0s 2ms/step - loss: 22272.0449 - mean_squared_error: 22272.0449
Epoch 142/500
150/150 [==============================] - 0s 2ms/step - loss: 22210.4043 - mean_squared_error: 22210.4043
Epoch 143/500
150/150 [==============================] - 0s 2ms/step - loss: 22128.6230 - mean_squared_error: 22128.6230
Epoch 144/500
150/150 [==============================] - 0s 2ms/step - loss: 22016.5762 - mean_squared_error: 22016.5762
Epoch 145/500
150/150 [=======================

150/150 [==============================] - 0s 2ms/step - loss: 14277.6123 - mean_squared_error: 14277.6123
Epoch 272/500
150/150 [==============================] - 0s 2ms/step - loss: 14271.0518 - mean_squared_error: 14271.0518
Epoch 273/500
150/150 [==============================] - 0s 2ms/step - loss: 14253.3770 - mean_squared_error: 14253.3770
Epoch 274/500
150/150 [==============================] - 0s 2ms/step - loss: 14281.3633 - mean_squared_error: 14281.3633
Epoch 275/500
150/150 [==============================] - 0s 2ms/step - loss: 14247.8623 - mean_squared_error: 14247.8623
Epoch 276/500
150/150 [==============================] - 0s 2ms/step - loss: 13998.4238 - mean_squared_error: 13998.4238
Epoch 277/500
150/150 [==============================] - 0s 2ms/step - loss: 14198.5605 - mean_squared_error: 14198.5605
Epoch 278/500
150/150 [==============================] - 0s 2ms/step - loss: 14031.0908 - mean_squared_error: 14031.0908
Epoch 279/500
150/150 [=======================

150/150 [==============================] - 0s 2ms/step - loss: 10693.7705 - mean_squared_error: 10693.7705
Epoch 406/500
150/150 [==============================] - 0s 2ms/step - loss: 10716.4014 - mean_squared_error: 10716.4014
Epoch 407/500
150/150 [==============================] - 0s 2ms/step - loss: 10626.3525 - mean_squared_error: 10626.3525
Epoch 408/500
150/150 [==============================] - 0s 2ms/step - loss: 10671.8975 - mean_squared_error: 10671.8975
Epoch 409/500
150/150 [==============================] - 0s 2ms/step - loss: 10521.8018 - mean_squared_error: 10521.8018
Epoch 410/500
150/150 [==============================] - 0s 2ms/step - loss: 10488.1348 - mean_squared_error: 10488.1348
Epoch 411/500
150/150 [==============================] - 0s 3ms/step - loss: 10561.7832 - mean_squared_error: 10561.7832
Epoch 412/500
150/150 [==============================] - 0s 3ms/step - loss: 10484.2402 - mean_squared_error: 10484.2402
Epoch 413/500
150/150 [=======================

## Decentralized method

In [63]:
y_pred = []
skuModels = {}

for i in skuSet:
 #one model for each item, fitted on training set
 model_i = ANN
 skuModels[i] = model_i.fit(X_dict[i]['train'],y_dict[i]['train'],batch_size=20,epochs=50,validation_split=0)

 #compute and concatenate prediction of the model i on item i
 y_pred += list(model_i.predict(X_dict[i]['test']))


#computing overall performance metrics on y_pred and y_test:
print('Decentralized method with linear regression R2:',round(r2_score(y_test, np.array(y_pred)),3))
print('Decentralized method with linear regression MSE:', round(mean_squared_error(y_test, np.array(y_pred)),3))

res['Decentralized(SA-DL)']=[r2_score(y_test, np.array(y_pred))]

Epoch 1/50
4/4 [==============================] - 0s 4ms/step - loss: 495.6539 - mean_squared_error: 495.6539
Epoch 2/50
4/4 [==============================] - 0s 5ms/step - loss: 489.1131 - mean_squared_error: 489.1131
Epoch 3/50
4/4 [==============================] - 0s 6ms/step - loss: 482.7991 - mean_squared_error: 482.7991
Epoch 4/50
4/4 [==============================] - 0s 6ms/step - loss: 477.0755 - mean_squared_error: 477.0755
Epoch 5/50
4/4 [==============================] - 0s 5ms/step - loss: 477.5544 - mean_squared_error: 477.5544
Epoch 6/50
4/4 [==============================] - 0s 4ms/step - loss: 474.6238 - mean_squared_error: 474.6238
Epoch 7/50
4/4 [==============================] - 0s 7ms/step - loss: 476.3922 - mean_squared_error: 476.3922
Epoch 8/50
4/4 [==============================] - 0s 5ms/step - loss: 476.0762 - mean_squared_error: 476.0762
Epoch 9/50
4/4 [==============================] - 0s 4ms/step - loss: 470.0449 - mean_squared_error: 470.0449
Epoch 10/5

Epoch 25/50
4/4 [==============================] - 0s 2ms/step - loss: 233.1932 - mean_squared_error: 233.1932
Epoch 26/50
4/4 [==============================] - 0s 3ms/step - loss: 230.9152 - mean_squared_error: 230.9152
Epoch 27/50
4/4 [==============================] - 0s 3ms/step - loss: 228.7887 - mean_squared_error: 228.7887
Epoch 28/50
4/4 [==============================] - 0s 2ms/step - loss: 226.5995 - mean_squared_error: 226.5995
Epoch 29/50
4/4 [==============================] - 0s 2ms/step - loss: 224.4979 - mean_squared_error: 224.4979
Epoch 30/50
4/4 [==============================] - 0s 2ms/step - loss: 222.3464 - mean_squared_error: 222.3464
Epoch 31/50
4/4 [==============================] - 0s 3ms/step - loss: 220.3827 - mean_squared_error: 220.3827
Epoch 32/50
4/4 [==============================] - 0s 3ms/step - loss: 218.2148 - mean_squared_error: 218.2148
Epoch 33/50
4/4 [==============================] - 0s 2ms/step - loss: 216.3299 - mean_squared_error: 216.3299
E

4/4 [==============================] - 0s 3ms/step - loss: 47.6560 - mean_squared_error: 47.6560
Epoch 25/50
4/4 [==============================] - 0s 2ms/step - loss: 47.4021 - mean_squared_error: 47.4021
Epoch 26/50
4/4 [==============================] - 0s 3ms/step - loss: 47.1486 - mean_squared_error: 47.1486
Epoch 27/50
4/4 [==============================] - 0s 2ms/step - loss: 46.8865 - mean_squared_error: 46.8865
Epoch 28/50
4/4 [==============================] - 0s 3ms/step - loss: 46.6234 - mean_squared_error: 46.6234
Epoch 29/50
4/4 [==============================] - 0s 3ms/step - loss: 46.3871 - mean_squared_error: 46.3871
Epoch 30/50
4/4 [==============================] - 0s 2ms/step - loss: 46.1210 - mean_squared_error: 46.1210
Epoch 31/50
4/4 [==============================] - 0s 2ms/step - loss: 45.8979 - mean_squared_error: 45.8979
Epoch 32/50
4/4 [==============================] - 0s 2ms/step - loss: 45.6480 - mean_squared_error: 45.6480
Epoch 33/50
4/4 [==============

4/4 [==============================] - 0s 3ms/step - loss: 442.9369 - mean_squared_error: 442.9369
Epoch 23/50
4/4 [==============================] - 0s 3ms/step - loss: 438.3309 - mean_squared_error: 438.3309
Epoch 24/50
4/4 [==============================] - 0s 3ms/step - loss: 433.6174 - mean_squared_error: 433.6174
Epoch 25/50
4/4 [==============================] - 0s 3ms/step - loss: 428.7652 - mean_squared_error: 428.7652
Epoch 26/50
4/4 [==============================] - 0s 3ms/step - loss: 424.2072 - mean_squared_error: 424.2072
Epoch 27/50
4/4 [==============================] - 0s 3ms/step - loss: 419.6402 - mean_squared_error: 419.6402
Epoch 28/50
4/4 [==============================] - 0s 3ms/step - loss: 415.0717 - mean_squared_error: 415.0717
Epoch 29/50
4/4 [==============================] - 0s 3ms/step - loss: 410.7590 - mean_squared_error: 410.7590
Epoch 30/50
4/4 [==============================] - 0s 4ms/step - loss: 406.4576 - mean_squared_error: 406.4576
Epoch 31/50
4

4/4 [==============================] - 0s 3ms/step - loss: 2053.4780 - mean_squared_error: 2053.4780
Epoch 19/50
4/4 [==============================] - 0s 4ms/step - loss: 1939.4878 - mean_squared_error: 1939.4878
Epoch 20/50
4/4 [==============================] - 0s 3ms/step - loss: 1912.0223 - mean_squared_error: 1912.0223
Epoch 21/50
4/4 [==============================] - 0s 3ms/step - loss: 1912.3955 - mean_squared_error: 1912.3955
Epoch 22/50
4/4 [==============================] - 0s 3ms/step - loss: 1907.6805 - mean_squared_error: 1907.6805
Epoch 23/50
4/4 [==============================] - 0s 3ms/step - loss: 1893.4130 - mean_squared_error: 1893.4130
Epoch 24/50
4/4 [==============================] - 0s 3ms/step - loss: 1920.0182 - mean_squared_error: 1920.0182
Epoch 25/50
4/4 [==============================] - 0s 3ms/step - loss: 1929.6526 - mean_squared_error: 1929.6526
Epoch 26/50
4/4 [==============================] - 0s 3ms/step - loss: 1907.7915 - mean_squared_error: 1907.

...

4/4 [==============================] - 0s 4ms/step - loss: 21518.1309 - mean_squared_error: 21518.1309
Epoch 30/50
4/4 [==============================] - 0s 3ms/step - loss: 21435.9004 - mean_squared_error: 21435.9004
Epoch 31/50
4/4 [==============================] - 0s 3ms/step - loss: 21461.5254 - mean_squared_error: 21461.5254
Epoch 32/50
4/4 [==============================] - 0s 3ms/step - loss: 21387.4414 - mean_squared_error: 21387.4414
Epoch 33/50
4/4 [==============================] - 0s 3ms/step - loss: 21292.7734 - mean_squared_error: 21292.7734
Epoch 34/50
4/4 [==============================] - 0s 2ms/step - loss: 21325.1055 - mean_squared_error: 21325.1055
Epoch 35/50
4/4 [==============================] - 0s 3ms/step - loss: 21329.8945 - mean_squared_error: 21329.8945
Epoch 36/50
4/4 [==============================] - 0s 3ms/step - loss: 21275.1641 - mean_squared_error: 21275.1641
Epoch 37/50
4/4 [==============================] - 0s 4ms/step - loss: 21227.1113 - mean_squ

4/4 [==============================] - 0s 2ms/step - loss: 340.3867 - mean_squared_error: 340.3867
Epoch 27/50
4/4 [==============================] - 0s 3ms/step - loss: 337.5180 - mean_squared_error: 337.5180
Epoch 28/50
4/4 [==============================] - 0s 3ms/step - loss: 334.5838 - mean_squared_error: 334.5838
Epoch 29/50
4/4 [==============================] - 0s 3ms/step - loss: 331.7875 - mean_squared_error: 331.7875
Epoch 30/50
4/4 [==============================] - 0s 3ms/step - loss: 328.8932 - mean_squared_error: 328.8932
Epoch 31/50
4/4 [==============================] - 0s 3ms/step - loss: 326.1183 - mean_squared_error: 326.1183
Epoch 32/50
4/4 [==============================] - 0s 4ms/step - loss: 323.2998 - mean_squared_error: 323.2998
Epoch 33/50
4/4 [==============================] - 0s 3ms/step - loss: 320.5733 - mean_squared_error: 320.5733
Epoch 34/50
4/4 [==============================] - 0s 3ms/step - loss: 317.8135 - mean_squared_error: 317.8135
Epoch 35/50
4

Epoch 50/50
4/4 [==============================] - 0s 4ms/step - loss: 329.3030 - mean_squared_error: 329.3030
Epoch 1/50
4/4 [==============================] - 0s 3ms/step - loss: 105.5589 - mean_squared_error: 105.5589
Epoch 2/50
4/4 [==============================] - 0s 3ms/step - loss: 104.5306 - mean_squared_error: 104.5306
Epoch 3/50
4/4 [==============================] - 0s 3ms/step - loss: 103.6272 - mean_squared_error: 103.6272
Epoch 4/50
4/4 [==============================] - 0s 5ms/step - loss: 102.7928 - mean_squared_error: 102.7928
Epoch 5/50
4/4 [==============================] - 0s 3ms/step - loss: 102.0725 - mean_squared_error: 102.0725
Epoch 6/50
4/4 [==============================] - 0s 4ms/step - loss: 101.3673 - mean_squared_error: 101.3673
Epoch 7/50
4/4 [==============================] - 0s 3ms/step - loss: 100.6674 - mean_squared_error: 100.6674
Epoch 8/50
4/4 [==============================] - 0s 5ms/step - loss: 100.0177 - mean_squared_error: 100.0177
Epoch 9/5

4/4 [==============================] - 0s 3ms/step - loss: 1480.8430 - mean_squared_error: 1480.8430
Epoch 47/50
4/4 [==============================] - 0s 3ms/step - loss: 1481.7987 - mean_squared_error: 1481.7987
Epoch 48/50
4/4 [==============================] - 0s 3ms/step - loss: 1481.6490 - mean_squared_error: 1481.6490
Epoch 49/50
4/4 [==============================] - 0s 2ms/step - loss: 1472.8605 - mean_squared_error: 1472.8605
Epoch 50/50
4/4 [==============================] - 0s 2ms/step - loss: 1462.9131 - mean_squared_error: 1462.9131
Epoch 1/50
4/4 [==============================] - 0s 3ms/step - loss: 465.7546 - mean_squared_error: 465.7546
Epoch 2/50
4/4 [==============================] - 0s 3ms/step - loss: 467.3126 - mean_squared_error: 467.3126
Epoch 3/50
4/4 [==============================] - 0s 3ms/step - loss: 466.4926 - mean_squared_error: 466.4926
Epoch 4/50
4/4 [==============================] - 0s 3ms/step - loss: 464.1264 - mean_squared_error: 464.1264
Epoch 5

Epoch 20/50
4/4 [==============================] - 0s 5ms/step - loss: 155.3221 - mean_squared_error: 155.3221
Epoch 21/50
4/4 [==============================] - 0s 5ms/step - loss: 153.6826 - mean_squared_error: 153.6826
Epoch 22/50
4/4 [==============================] - 0s 5ms/step - loss: 152.0742 - mean_squared_error: 152.0742
Epoch 23/50
4/4 [==============================] - 0s 5ms/step - loss: 150.5497 - mean_squared_error: 150.5497
Epoch 24/50
4/4 [==============================] - 0s 5ms/step - loss: 148.9741 - mean_squared_error: 148.9741
Epoch 25/50
4/4 [==============================] - 0s 5ms/step - loss: 147.4178 - mean_squared_error: 147.4178
Epoch 26/50
4/4 [==============================] - 0s 5ms/step - loss: 145.9537 - mean_squared_error: 145.9537
Epoch 27/50
4/4 [==============================] - 0s 5ms/step - loss: 144.3956 - mean_squared_error: 144.3956
Epoch 28/50
4/4 [==============================] - 0s 5ms/step - loss: 142.9470 - mean_squared_error: 142.9470
E

# Results

In [64]:
res

,Centralized(DL),Decentralized(DL),Centralized(SA-DL),Decentralized(SA-DL)
R2,0.543218,0.662547,0.659943,0.655719


It is evident that the decentralization method without seasonality aggregation has the most accuracy in predicting the weekly sales. 